# BFX Workshop Week 9 - Parsing and Filtering with Python

## Setup

For this module we'll be working with a somatic exome VCF file created by the Mutect variant caller with some basic filtering already done. Let's create a working directory and download this file.

Note that some of the following dropbox links may break in the future; all files are mirrored at [box]( https://wustl.app.box.com/folder/149874570252). `wget` will not work for box links, so you will have to follow the link in your browser and download the files that way.  

In [ ]:
!echo $PWD
!mkdir -p $PWD/bfx_workshop_week_09
!wget https://www.dropbox.com/s/cwktbkj72va2dci/mutect.filtered.vcf.gz -O $PWD/bfx_workshop_week_09/mutect.filtered.vcf.gz

We will also need the reference and tumor bam files used previously.

In [ ]:
!docker run -v $PWD/bfx_workshop_week_09:/staging mgibio/data_downloader:0.1.0 gsutil -m cp gs://analysis-workflows-example-data/somatic_inputs/hla_and_brca_genes.fa /staging
!docker run -v $PWD/bfx_workshop_week_09:/staging mgibio/data_downloader:0.1.0 gsutil -m cp gs://analysis-workflows-example-data/somatic_inputs/hla_and_brca_genes.fa.fai /staging
!wget https://www.dropbox.com/s/7k92tp014f6zzg4/tumor.bam -O $PWD/bfx_workshop_week_09/tumor.bam
!wget https://www.dropbox.com/s/mogij2hxuxsur1f/tumor.bam.bai -O $PWD/bfx_workshop_week_09/tumor.bam.bai

In order to speed up later steps, let's pull the Docker containers we will be using

In [ ]:
!docker pull mgibio/bam_readcount_helper-cwl:1.1.1
!docker pull quay.io/biocontainers/vt:0.57721--hf74b74d_1
!docker pull griffithlab/vatools:5.0.1

# Intro to VCF
The [Variant Call Format (VCF)](https://samtools.github.io/hts-specs/VCFv4.2.pdf) is a text file format. It contains meta-information header lines and data lines with information about a position in the genome. It can optionally contain genotype information on samples for each position.

## Types of data encoded
### FILTER
Filters that have been applied to a data line

### INFO
Per-variant data

### FORMAT
Per-sample data

## Meta-information header lines
Provide information about how each field is encoded.
### File format
`##fileformat=VCFv4.2`

### FILTER
Each FILTER header describes one specific FILTER. The header includes the ID of the FILTER and a description.

`##FILTER=<ID=ID,Description="description">`

e.g.:

`##FILTER=<ID=PASS,Description="All filters passed">`

### INFO
Each INFO header describes one specific INFO field. The header includes the ID of the INFO field, a Number value describing how many data points will be encoded in the field, a Type describing the data type of each data point in this field, and a Description. Source and Version are optional but can be used to descibe which tool added this field and the version of the tool.

`##INFO=<ID=ID,Number=number,Type=type,Description="description",Source="source",Version="version">`

e.g.:

`##INFO=<ID=MBQ,Number=R,Type=Integer,Description="median base quality">`

### FORMAT
Each FORMAT header descibes one specific per-sample field. The header includes the ID of the per-sample field a Number value describing how many data points will be encoded in the field, a Type describing the data type of each data point in this field, and a Description.

`##FORMAT=<ID=ID,Number=number,Type=type,Description="description">`

e.g.:

`##FORMAT=<ID=AF,Number=A,Type=Float,Description="Allele fractions of alternate alleles in the tumor">`

### Header line (starts with CHROM)
The Header line describes each column of the data lines, which immediately follow this line.

`#CHROM POS ID REF ALT QUAL FILTER INFO`

If your VCF contains per-sample information, the header line will also include a FORMAT field, followed for one field for each sample.

`#CHROM POS ID REF ALT QUAL FILTER INFO FORMAT Exome_Normal Exome_Tumor`

## Example
```
##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##INFO=<ID=MBQ,Number=R,Type=Integer,Description="median base quality">
##INFO=<ID=MMQ,Number=R,Type=Integer,Description="median mapping quality">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=AD,Number=R,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed">
##FORMAT=<ID=AF,Number=A,Type=Float,Description="Allele fractions of alternate alleles in the tumor">
##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Approximate read depth (reads with MQ=255 or with bad mates are filtered)">
#CHROM    POS    ID    REF    ALT    QUAL    FILTER    INFO    FORMAT    Exome_Normal    Exome_Tumor
chr17	2364457	.	T	G	.	PASS	MBQ=30,30;MMQ=60,60	GT:AD:AF:DP	0/0:85,0:0.011:85	0/1:43,58:0.573:101
```

# Preprocessing VCF - adding readcounts using bam-readcount and VAtools

## Splitting multialleleic sites using vt decompose

Our VCF might contain variants with multiple alt alleles. In these cases the ALT field of the VCF will have multiple alt alleles in it. Take for example this variant:
```
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	Exome_Normal	Exome_Tumor
chr17	3017916	.	CGTGT	C,CGT	.	germline;multiallelic;normal_artifact	AS_FilterStatus=weak_evidence|SITE;AS_SB_TABLE=31,0|3,0|9,0;DP=48;ECNT=1;GERMQ=1;MBQ=30,30,30;MFRL=0,0,0;MMQ=60,60,60;MPOS=49,31;NALOD=0.710,-7.297e+00;NLOD=2.65,-6.178e+00;POPAF=6.00,6.00;RPA=16,14,15;RU=GT;STR;STRQ=93;TLOD=6.76,12.45	GT:AD:AF:DP:F1R2:F2R1:SB	0/0:9,0,3:0.066,0.271:12:0,0,0:8,0,3:9,0,3,0	0/1/2:22,3,6:0.117,0.205:31:0,0,0:22,3,6:22,0,9,0
```
This might happen if both chromsomes have a mutation at the same position, but the exact mutation differs between the two chromosomes. It might also happen if there is a subclonal mutation in some tumor cells. It might also just be an artifact.

It is usually easier to process a VCF if these sort of variants are preprocessed to split up multi-allelic sites since some information is encoded on a per-allele basis (e.g., per-allele depth, per-allele VAF). 

vt decompose is part of the [vt tool package](https://genome.sph.umich.edu/wiki/Vt) and available on quay container at `quay.io/biocontainers/vt:0.57721--hf74b74d_1`.

In [ ]:
!docker run -v $PWD/bfx_workshop_week_09:/data -it quay.io/biocontainers/vt:0.57721--hf74b74d_1 vt decompose /data/mutect.filtered.vcf.gz -s -o /data/mutect.filtered.decomposed.vcf.gz 

After running vt decompose the above variant is split up into two lines and looks like this:
```
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	Exome_Normal	Exome_Tumor
chr17	3017916	.	CGTGT	C	.	germline;multiallelic;normal_artifact	AS_FilterStatus=weak_evidence|SITE;AS_SB_TABLE=31,0|3,0|9,0;DP=48;ECNT=1;GERMQ=1;MBQ=30,30;MFRL=0,0;MMQ=60,60;MPOS=49;NALOD=0.71;NLOD=2.65;POPAF=6;RPA=16,14;RU=GT;STR;STRQ=93;TLOD=6.76;OLD_MULTIALLELIC=chr17:3017916:CGTGT/C/CGT	GT:AD:AF:DP:F1R2:F2R1:SB	0/0:9,0:0.066:12:0,0:8,0:9,0,3,0	0/1/.:22,3:0.117:31:0,0:22,3:22,0,9,0
chr17	3017916	.	CGTGT	CGT	.	germline;multiallelic;normal_artifact	AS_FilterStatus=weak_evidence|SITE;AS_SB_TABLE=31,0|3,0|9,0;DP=48;ECNT=1;GERMQ=1;MBQ=30,30;MFRL=0,0;MMQ=60,60;MPOS=31;NALOD=-7.297;NLOD=-6.178;POPAF=6;RPA=16,15;RU=GT;STR;STRQ=93;TLOD=12.45;OLD_MULTIALLELIC=chr17:3017916:CGTGT/C/CGT	GT:AD:AF:DP:F1R2:F2R1:SB	0/0:9,3:0.271:12:0,0:8,3:9,0,3,0	0/./1:22,6:0.205:31:0,0:22,6:22,0,9,0
```

## bam-readcount
Some variant callers will already output read depth and allelic depths but this is useful in cases where this information is not already present in the VCF. This is also useful if you run RNAseq on top of somatic variant calling to add RNA coverage information to your VCF.

We will be using the `mgibio/bam_readcount_helper-cwl:1.1.1` docker container to run bam-readcount. This Docker image already has bam-readcount installed and it also contains a script that will take care of creating a region list from your VCF, which is a required input to bam-readcount.

### Required inputs
- vcf
- sample name
- reference fasta
- bam file
- output file prefix
- output directory

### Tip
To quickly find sample names in a VCF, try searching the file for `#CHROM`. This is guaranteed to be part of the header, and the names of any samples included in the file will be present at the end of this line. 
Normally we would use `grep` to search through a file, but VCF files are often stored `gzip`-compressed to save space. Luckily, `zgrep` is standard alongside grep, and can be used to search compressed files in place, without needing to manually unzip, search, then re-zip. Similar counterparts exist to other standard tools, such as `cat`/`zcat` and `diff`/`zdiff`.

In [ ]:
!zgrep '#CHROM' bfx_workshop_week_09/mutect.filtered.decomposed.vcf.gz

In [ ]:
!docker run -v $PWD/bfx_workshop_week_09:/data -it mgibio/bam_readcount_helper-cwl:1.1.1 python /usr/bin/bam_readcount_helper.py /data/mutect.filtered.decomposed.vcf.gz Exome_Tumor /data/hla_and_brca_genes.fa /data/tumor.bam Exome_Tumor /data

## VAtools
[VAtools](http://www.vatools.org) is a python package that provides a suite of tools that help with processing VCF annotations. We will be using the [vcf-readcount-annotator tool](https://vatools.readthedocs.io/en/latest/vcf_readcount_annotator.html) included with VAtools to write the readcounts calculated in the previous step to our VCF. VAtools is available as a Docker image at `griffithlab/vatools:4.1.0`.

In [ ]:
!docker run -v $PWD/bfx_workshop_week_09:/data -it griffithlab/vatools:5.0.1 vcf-readcount-annotator /data/mutect.filtered.decomposed.vcf.gz /data/Exome_Tumor_Exome_Tumor_bam_readcount_snv.tsv DNA -t snv -s Exome_Tumor -o /data/mutect.filtered.decomposed.readcount_snvs.vcf.gz
!docker run -v $PWD/bfx_workshop_week_09:/data -it griffithlab/vatools:5.0.1 vcf-readcount-annotator /data/mutect.filtered.decomposed.readcount_snvs.vcf.gz /data/Exome_Tumor_Exome_Tumor_bam_readcount_indel.tsv DNA -s Exome_Tumor -t indel -o /data/mutect.filtered.decomposed.readcount_snvs_indel.vcf.gz

# Parsing VCFs in Python

## PyVCF vs VCFPy

[PyVCF](https://pyvcf.readthedocs.io/en/latest/) is the "original" Python VCF parser. Unfortunately, it doesn't appear to be maintained anymore and is incompatible with newer version of setuptools, leading to installation errors. It also doesn't support modifying VCF entries very well. [VCFPy](https://vcfpy.readthedocs.io/en/stable/) was created to solve this problem. For that reason we'll be using VCFPy for the next tasks.

First, we need to ensure that the `vcfpy` package is installed.

In [ ]:
!pip install vcfpy pysam

### Reading in a VCF and exploring its contents

In [ ]:
import vcfpy

Create the VCF reader object from your VCF path

In [ ]:
vcf_reader = vcfpy.Reader.from_path("bfx_workshop_week_09/mutect.filtered.decomposed.readcount_snvs_indel.vcf.gz")

Which samples are in your VCF?

In [ ]:
vcf_reader.header.samples.names

Which FILTERS are defined in the VCF header?

In [ ]:
vcf_reader.header.filter_ids()

Similar methods `info_ids` and `format_ids` exist for the INFO and FORMAT fields.

Get information for a specific INFO header

In [ ]:
vcf_reader.header.get_info_field_info('DP')

Get information for each variant

In [ ]:
vcf_reader = vcfpy.Reader.from_path("bfx_workshop_week_09/mutect.filtered.decomposed.readcount_snvs_indel.vcf.gz")
for entry in vcf_reader:
    #Get all FILTER fields applied to this variant 
    print(entry.FILTER)
    #Get the VAFs of a variant
    print(entry.call_for_sample['Exome_Tumor'].data)
    break

After your're done with all processing, you will need to close the file.

In [ ]:
vcf_reader.close()

### Filtering a VCF

Let's create a filtered VCF so that only variants with a `PASS` filter and a VAF over 0.25 will remain. 

In [ ]:
import vcfpy
vcf_reader = vcfpy.Reader.from_path("bfx_workshop_week_09/mutect.filtered.decomposed.readcount_snvs_indel.vcf.gz")
vcf_writer = vcfpy.Writer.from_path("bfx_workshop_week_09/mutect.filtered.decomposed.readcount_snvs_indel.pass_vaf_filtered.vcf.gz", vcf_reader.header)
for entry in vcf_reader:
    if 'PASS' in entry.FILTER and entry.call_for_sample['Exome_Tumor'].data['AF'][0] > 0.25:
        vcf_writer.write_record(entry)
vcf_reader.close()
vcf_writer.close()

In [ ]:
!zgrep -v '#' bfx_workshop_week_09/mutect.filtered.decomposed.readcount_snvs_indel.vcf.gz | wc -l

In [ ]:
!zgrep -v '#' bfx_workshop_week_09/mutect.filtered.decomposed.readcount_snvs_indel.pass_vaf_filtered.vcf.gz | wc -l

### Creating a human-readable TSV file

VCFs can often be hard to read since a lot of information is presented in a condensed manner. Let's output some of its information in an easier understandable TSV file.

In [ ]:
import vcfpy
import csv

vcf_reader = vcfpy.Reader.from_path("bfx_workshop_week_09/mutect.filtered.decomposed.readcount_snvs_indel.pass_vaf_filtered.vcf.gz")
with open("bfx_workshop_week_09/mutect.filtered.decomposed.readcount_snvs_indel.pass_vaf_filtered.tsv", 'w') as out_fh:
    headers = ['CHROM', 'POS', 'REF', 'ALT', 'FILTER', 'DEPTH', 'VAF']
    tsv_writer = csv.DictWriter(out_fh, delimiter = '\t', fieldnames = headers)
    tsv_writer.writeheader()
    for entry in vcf_reader:
        out = {
            'CHROM': entry.CHROM,
            'POS': entry.POS,
            'REF': entry.REF,
            'ALT': ','.join( [alt.serialize() for alt in entry.ALT] ),
            'FILTER': ','.join(entry.FILTER),
            'DEPTH': entry.call_for_sample['Exome_Tumor'].data['DP'],
            'VAF': ','.join( [str(vaf) for vaf in entry.call_for_sample['Exome_Tumor'].data['AF']] )
        }
        tsv_writer.writerow(out)
vcf_reader.close()

In [ ]:
!cat bfx_workshop_week_09/mutect.filtered.decomposed.readcount_snvs_indel.pass_vaf_filtered.tsv

# Python Tips

## REPL
If you're in a hurry, on an unfamiliar machine, or just don't like Jupyter, but want a similar way to quickly try out some code and see results, try out the python REPL!
- This stands for read-eval-print-loop, and functions much like the embedded code snippets in the notebook above, or like REPLs packaged with other languages
- Invoke with `python -i`
- Quit with `exit()`
- Scripts can also be launched interactively with `python -i <script.py>`, and upon exit (whether normally or due to an error), the state is passed to the REPL, and you can run functions from the script, examine variable values, etc

## Useful bioinformatics packages
- vcfpy
- pysam- samtools wrapper
- pyyaml- work with YAML files (CWL, the language that our managed workflow are written in, is actually YAML)
- pandas & numpy- packages for efficiently working with big data
- biopython- a vast collection of bioinformatics tool. Also useful for parsing and writing FASTA files.

# Homework
- Download the normal bams from here: https://www.dropbox.com/s/jhru5qu4bxcmfwq/normal.bam, https://www.dropbox.com/s/4gimazw35rcxr8x/normal.bam.bai
- Calculate readcounts for the normal sample and add them to the `mutect.filtered.decomposed.pass_vaf_filtered.vcf.gz` file
- Filter the output VCF from the above step to exclude variants where the normal VAF is higher than 0.02
- Amend the TSV-creation script to also output the normal VAF on top of the other information
- Read Chapter 1 of the the [VCF specification document](https://samtools.github.io/hts-specs/VCFv4.2.pdf) to familiarize yourself further with the VCF format. What are the differences between the FILTER, INFO, and FORMAT fields (i.e., when would you use which field)? What do the Number types A, R, G, and . mean? Make a list of additional questions you might have about the VCF format.